In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append('../scripts')
import Baseline_classifiers as bc

In [2]:
organisms = ["E.Coli"]#"Drosophila.Melanogaster", "Homo.Sapiens"

dfs = {}
usage_biases = {}

def group_codons(sequence):
    return [''.join(sequence[i:i+3]) for i in range(0, len(sequence), 3)]

for organism in organisms:
    dfs[organism] = pd.read_pickle(f"../data/{organism}/cleanedData.pkl")
    dfs[organism]['codons'] = dfs[organism]['sequence'].apply(group_codons)
    usage_biases[organism] = pd.read_pickle(f"../data/{organism}/usageBias.pkl")

display(dfs[organisms[0]].head())
display(usage_biases[organisms[0]])

,id,description,sequence,translation,seguid,codons
0,lcl|U00096.3_cds_AAC73112.1_1,lcl|U00096.3_cds_AAC73112.1_1 [gene=thrL] [loc...,"(A, T, G, A, A, A, C, G, C, A, T, T, A, G, C, ...","(M, K, R, I, S, T, T, I, T, T, T, I, T, I, T, ...",QuAqLUJXXhVXIJRPc+T7iMi0UqE,"[ATG, AAA, CGC, ATT, AGC, ACC, ACC, ATT, ACC, ..."
1,lcl|U00096.3_cds_AAC73113.1_2,lcl|U00096.3_cds_AAC73113.1_2 [gene=thrA] [loc...,"(A, T, G, C, G, A, G, T, G, T, T, G, A, A, G, ...","(M, R, V, L, K, F, G, G, T, S, V, A, N, A, E, ...",/p+3Jdgat4Fq0w2rqqay4xg8Bs4,"[ATG, CGA, GTG, TTG, AAG, TTC, GGC, GGT, ACA, ..."
2,lcl|U00096.3_cds_AAC73114.1_3,lcl|U00096.3_cds_AAC73114.1_3 [gene=thrB] [loc...,"(A, T, G, G, T, T, A, A, A, G, T, T, T, A, T, ...","(M, V, K, V, Y, A, P, A, S, S, A, N, M, S, V, ...",scFdp6oP+pW5HnvUww8MSi93ntM,"[ATG, GTT, AAA, GTT, TAT, GCC, CCG, GCT, TCC, ..."
3,lcl|U00096.3_cds_AAC73115.1_4,lcl|U00096.3_cds_AAC73115.1_4 [gene=thrC] [loc...,"(A, T, G, A, A, A, C, T, C, T, A, C, A, A, T, ...","(M, K, L, Y, N, L, K, D, H, N, E, Q, V, S, F, ...",EkFurFHdk7l6qKQ+5xRiBUhWM9U,"[ATG, AAA, CTC, TAC, AAT, CTG, AAA, GAT, CAC, ..."
5,lcl|U00096.3_cds_AAC73117.1_6,lcl|U00096.3_cds_AAC73117.1_6 [gene=yaaA] [loc...,"(A, T, G, C, T, G, A, T, T, C, T, T, A, T, T, ...","(M, L, I, L, I, S, P, A, K, T, L, D, Y, Q, S, ...",vJJ0yR31YORqwI12U79SgItYU3U,"[ATG, CTG, ATT, CTT, ATT, TCA, CCT, GCG, AAA, ..."


{'M': {'ATG': 1.0},
 'K': {'AAA': 0.7697623206014124, 'AAG': 0.23023767939858758},
 'R': {'CGC': 0.4038268652110177,
  'CGA': 0.06237173057634442,
  'CGT': 0.3853406372965049,
  'AGG': 0.019365789676722475,
  'CGG': 0.09391250675102229,
  'AGA': 0.03518247048838824},
 'I': {'ATT': 0.5083169031684763,
  'ATC': 0.4226625197856211,
  'ATA': 0.06902057704590264},
 'S': {'AGC': 0.27845605080764435,
  'TCA': 0.12152364933445466,
  'TCT': 0.1468839718121973,
  'AGT': 0.14924745526781313,
  'TCG': 0.15393092248354262,
  'TCC': 0.14995795029434794},
 'T': {'ACC': 0.439674039580908,
  'ACA': 0.1286767559177338,
  'ACG': 0.26575087310826545,
  'ACT': 0.16589833139309274},
 'G': {'GGT': 0.33815828251456553,
  'GGC': 0.40754006937045184,
  'GGG': 0.1490104509079368,
  'GGA': 0.10529119720704586},
 'N': {'AAC': 0.5532204543520803, 'AAT': 0.44677954564791966},
 'A': {'GCG': 0.35725717099593746,
  'GCA': 0.2125446263695679,
  'GCC': 0.27000932097571273,
  'GCT': 0.16018888165878195},
 '*': {'TGA': 0.2

# Baseline Klassifikatoren

Berechne den Fehler Rate $E$  $$E= {F \over G} $$
$F$: Anzahl falsch klassifizierter Aminosäuren
über alle Sequenzen eines betrachteten Organismus \
$G$: Gesamtanzahl der zu klassifizierenden Aminosäuren 
über alle Sequenzen eines betrachteten Organismus



In [3]:
def test_classifier(classifier, organism):
    amino_seq = dfs[organism]['translation'].apply(lambda seq: list(seq))
    pred_codons = classifier.predict_codons(amino_seq)
    true_codons = dfs[organism]['codons']
    error_rate = classifier.calc_error_rate(true_codons, pred_codons)
    error_rate_per_amino_acid = classifier.calc_amino_acid_error_rate(amino_seq, true_codons, pred_codons)
    print(f"Organismus {organism} - Error Rate: {error_rate}")
    print("Error Rates pro Aminosäure:")
    print(error_rate_per_amino_acid)

## Baseline Klassifikator mit ungewichtetem Zufall

In [4]:
print("Ungewichteter Baseline Klassifikator")
for organism in organisms:
    seed = 42
    unweighted_bc = bc.Unweighted_Baseline_Classifier(seed)
    test_classifier(unweighted_bc, organism)

Ungewichteter Baseline Klassifikator
Organismus E.Coli - Error Rate: 0.6457996806107671
Error Rates pro Aminosäure:
{'A': 0.7492129930884086, 'R': 0.8353367795694777, 'N': 0.5009572024164043, 'D': 0.5013979193758128, 'C': 0.49358184059757776, 'Q': 0.5004745273707387, 'E': 0.5013794906049429, 'G': 0.7484981070481286, 'H': 0.498812351543943, 'I': 0.6685318374940991, 'L': 0.8340064397786228, 'K': 0.49996203204495404, 'M': 0.0, 'F': 0.4977981102227543, 'P': 0.7488437334142088, 'S': 0.8330771684598208, 'T': 0.7483430345362825, 'W': 0.0, 'Y': 0.49902981125418944, 'V': 0.7487072934620924, '*': 0.6793774319066148}


## Baseline Klassifikator mit Usage Bias gewichtetem Zufall

In [5]:
print("Usage Bias Gewichteter Baseline Klassifikator")
for organism in organisms:
    seed = 42
    weighted_bc = bc.Bias_Weighted_Classifier(usage_biases[organism], seed)
    test_classifier(weighted_bc, organism)

Usage Bias Gewichteter Baseline Klassifikator
Organismus E.Coli - Error Rate: 0.5833700327328002
Error Rates pro Aminosäure:
{'A': 0.7292256555459805, 'R': 0.6729881953552966, 'N': 0.4942567855015741, 'D': 0.4707087126137841, 'C': 0.500471390238596, 'Q': 0.4532116012451598, 'E': 0.42729358464263933, 'G': 0.6849311963002426, 'H': 0.49190914489311166, 'I': 0.5586487462164339, 'L': 0.6956535430591311, 'K': 0.3536525172754195, 'M': 0.0, 'F': 0.4900166745051092, 'P': 0.6453104860110699, 'S': 0.8162282864019952, 'T': 0.6955064027939465, 'W': 0.0, 'Y': 0.49032751220085846, 'V': 0.7273746842018275, '*': 0.490272373540856}


## Baseline Klassifikator der immer das nach Usage Bias wahrscheinlichste Codon wählt

In [6]:
print("Usage Bias Maximum Baseline Klassifikator")
for organism in organisms:
    seed = 42
    max_weighted_bc = bc.Max_Bias_Baseline_Classifier(usage_biases[organism], seed)
    test_classifier(max_weighted_bc, organism)

Usage Bias Maximum Baseline Klassifikator
Organismus E.Coli - Error Rate: 0.4815799655540969
Error Rates pro Aminosäure:
{'A': 0.6427428290040625, 'R': 0.5961731347889824, 'N': 0.44677954564791966, 'D': 0.37460988296488945, 'C': 0.4411487417506708, 'Q': 0.3462341507858173, 'E': 0.3079168239769988, 'G': 0.5924599306295482, 'H': 0.4308565914489311, 'I': 0.4916830968315237, 'L': 0.5002007510450863, 'K': 0.23023767939858758, 'M': 0.0, 'F': 0.4268887083671812, 'P': 0.46976647206005007, 'S': 0.7215439491923556, 'T': 0.560325960419092, 'W': 0.0, 'Y': 0.43058740518609984, 'V': 0.6304748187849739, '*': 0.35304798962386513}
